## Table of Contents
  - [k-Nearest Neighbors (k-NN)](#k-Nearest-Neighbors-%28k-NN%29)
- [Implementation](#Implementation)
  - [Visualization](#Visualization)
- [🏠 Home](../../../../welcomePage.ipynb)

## k-Nearest Neighbors (k-NN)

**k-Nearest Neighbors (k-NN)** is a simple, easy-to-implement supervised machine learning algorithm that can be used for both classification and regression tasks. The k-NN algorithm assumes that similar things exist in close proximity. In other words, similar things are near to each other.

### Concept

k-NN works by finding the distances between a query and all the examples in the data, selecting the specified number `k` nearest neighbors (top k examples with the smallest distances), and performing a majority vote (for classification) or averaging (for regression) on these neighbors.

### Steps Involved

1. **Calculate Distance:** Compute the distance between the point in question and every other point in the dataset.
2. **Find Nearest Neighbors:** Identify the `k` closest points based on these calculated distances.
3. **Vote for Labels:** For classification, the label of the new point is determined by a majority vote of the `k` nearest neighbors. For regression, it is the average of the values of the `k` nearest neighbors.

### Distance Metrics

The choice of distance metric depends on the type of data:
- **Euclidean Distance:** Most common for continuous data.
- **Manhattan Distance:** Used for ordinal or vector data.
- **Hamming Distance:** Used for categorical data.

### Hyperparameters

- **Number of Neighbors (k):** Determines how many neighbors will influence the classification or regression. Choosing the right `k` is crucial for performance.
- **Weight Function:** Typically, neighbors are weighted equally. However, sometimes it might be beneficial to weight neighbors differently based on distance.

### Choosing the Right `k`

- **Too small `k`:** Makes the model sensitive to noise in the dataset.
- **Too large `k`:** Makes the boundaries between classes less distinct.

### Applications

k-NN is widely used in:
- Medical diagnosis,
- Pattern recognition,
- Recommendation systems (e.g., suggesting similar products).


<img src="knn_illustration.png" alt="Zhang, Yifei & Chen, Zongsen & Zhang, Shaobo & Wang, Junzhuan. (2023). Color Sensing and Image Reconstruction Using Intelligent Machine Learning Algorithm with PINIP Radial Junction Imager. Symmetry. 15. 2127. 10.3390/sym15122127. " title="Title">


# Implementation

### Import Libraries

**Press ▶ to import the libraries.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

print("✅ Libraries are imported.")

### Import data

**Press ▶ to load the data.**

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# List all .csv and Excel files in the current directory
supported_extensions = ['.csv', '.xlsx', '.xls']
files = [f for f in os.listdir('./data/') if any(f.endswith(ext) for ext in supported_extensions)]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=files,
    description='Files:',
    disabled=False,
)

# Create a button widget
button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='Click to select file',
    icon='check'
)

# Output widget to display messages
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with output:
        clear_output()
        selected_file = dropdown.value
        global data
        if selected_file.endswith('.csv'):
            data = pd.read_csv('./data/'+selected_file)
        elif selected_file.endswith(('.xlsx', '.xls')):
            data = pd.read_excel('./data/'+selected_file)
        print(f"File '{selected_file}' uploaded as data.")

# Attach the function to the button widget
button.on_click(on_button_click)

# Display the dropdown, button widgets, and initial message within the output widget
with output:
    print("Please select a file from the dropdown and click 'Select'.")
display(output)
display(dropdown)
display(button)

**Press ▶ to display the data.**

In [ ]:
display(data.head())
print("Loading pair plot, please wait...")
sns.pairplot(data=data,hue=data.columns[-1],diag_kind="hist")

### Select a Target Column

**Press ▶ to specify the target column.**

In [ ]:
import ipywidgets as widgets
import pandas as pd

# Create a Dropdown widget for column selection
dropdown = widgets.Dropdown(
    options=data.columns.tolist(),
    value=data.columns[0],
    description='Select Target Column:',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create a Button widget
button = widgets.Button(
    description='Select',
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to select the target column as the last column',
    icon='check'  # FontAwesome icon names (without 'fa-')
)

# Create an Output widget for displaying messages
output = widgets.Output()

# Function to handle button click that rearranges the DataFrame
def on_button_clicked(b):
    with output:
        output.clear_output()
        global data
        # Get the selected column name
        selected_column = dropdown.value
        # Reorder the DataFrame columns
        new_columns = [col for col in data.columns if col != selected_column] + [selected_column]
        data = data[new_columns]
        print(f"Column '{selected_column}' has been moved to the last position.")

# Link the button click event to the function
button.on_click(on_button_clicked)

# Display the widgets and output
display(widgets.VBox([dropdown, button, output]))


### Data Preprocessing

**Press ▶ to handle nulls, process categorical values, and normalize data.**

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df = data.copy()

# Widgets for selecting operations and methods
fill_method_dropdown = widgets.Dropdown(
    options=[('None', None), ('Zero', 'zero'), ('Mean', 'mean'), ('Median', 'median')],
    value=None,
    description='Fill Method:',
)

remove_nulls_checkbox = widgets.Checkbox(value=False, description='Remove Nulls')
encode_categorical_checkbox = widgets.Checkbox(value=False, description='Encode Categorical')
normalize_data_checkbox = widgets.Checkbox(value=False, description='Normalize Data')

# Button to apply selected operations
apply_button = widgets.Button(description='Apply All', button_style='info')

# Button to display data
show_data_button = widgets.Button(description='Show Data')

# Output area
output = widgets.Output()

def apply_operations():
    global data
    with output:
        output.clear_output()
        # Fill missing values based on selected method
        if fill_method_dropdown.value:
            if fill_method_dropdown.value == 'zero':
                data = data.fillna(0)
            elif fill_method_dropdown.value in ['mean', 'median']:
                # Apply fill method only to numeric columns
                numeric_cols = data.select_dtypes(include=np.number).columns
                for column in numeric_cols:
                    if data[column].sum() != 0:
                        if fill_method_dropdown.value == 'mean':
                            mean_value = data[column].mean()
                            data[column] = data[column].fillna(mean_value)
                        elif fill_method_dropdown.value == 'median':
                            median_value = data[column].median()
                            data[column] = data[column].fillna(median_value)
            print(f"Missing values filled with {fill_method_dropdown.value}.")

        if remove_nulls_checkbox.value:
            data = data.dropna()  # Remove remaining null values
            print("Remaining null values removed.")
        if encode_categorical_checkbox.value:
            # Apply label encoding to categorical columns
            label_encoder = LabelEncoder()
            categorical_cols = data.select_dtypes(include=['object', 'category']).columns
            for col in categorical_cols:
                data[col] = label_encoder.fit_transform(data[col].astype(str))
            print("Categorical data encoded using label encoding.")
        if normalize_data_checkbox.value:
            # Normalize numeric columns using MinMaxScaler
            scaler = MinMaxScaler()
            numeric_cols = data.select_dtypes(include=np.number).columns
            data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
            print("Data normalized.")

def show_data(b):
    with output:
        output.clear_output()
        display(data.head())  # Show the head of the DataFrame

apply_button.on_click(lambda b: apply_operations())
show_data_button.on_click(show_data)

# Layout the widgets
widgets.VBox([
    widgets.Label('Select Fill Method and Operations:'),
    fill_method_dropdown,
    remove_nulls_checkbox,
    encode_categorical_checkbox,
    normalize_data_checkbox,
    apply_button,
    show_data_button,
    output
])


### Split data into training and testing

**Press ▶ to split the data into training and testing sets.**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display, clear_output

X = data.iloc[:, :-1]  # all rows, all columns except the last one
y = data.iloc[:, -1]   # all rows, just the last column
# Function to split data and display the shape of the splits

X_train, X_test, y_train, y_test = None, None, None, None

def split_data(button):
    global X_train, X_test, y_train, y_test  # Declare the use of global variables
    test_size = test_size_slider.value
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Display the output
    with out:
        clear_output(wait=True)
        print(f"Training set: {X_train.shape[0]} samples")
        print(f"Test set: {X_test.shape[0]} samples")

# Create output widget
out = widgets.Output()


# Create slider for test size
test_size_slider = widgets.FloatSlider(
    value=0.25,  # Default split 75%-25%
    min=0.1,
    max=0.9,
    step=0.05,
    description='Test Size:',
    readout_format='.2f',  # Display format
)

# Create an Apply button
apply_button = widgets.Button(description="Apply Changes")

# Set up button click event to trigger the data split
apply_button.on_click(split_data)

# Organize widgets in a vertical box
widgets_box = widgets.VBox([test_size_slider, apply_button, out])

# Display the widgets
display(widgets_box)

### kNN

### Key Parameters in k-Nearest Neighbors (kNN)

The k-Nearest Neighbors (kNN) algorithm is a simple, easy-to-implement supervised machine learning algorithm that can be used to solve both classification and regression problems. However, the choice of parameters and distance measures can significantly affect the performance of the model. Here are key parameters:

#### 1. Number of Neighbors (`n_neighbors`)

The `n_neighbors` parameter specifies the number of neighbors to use for kneighbors queries. The choice of `n_neighbors` significantly influences the performance of the model. If the number is too small, the model might be too complex and sensitive to noise in the dataset; this leads to overfitting. If it is too large, the neighborhood may include too many points from other classes, leading to underfitting. Therefore, finding a good balance for `n_neighbors` is crucial for the model's performance.

#### 2. Power Parameter (`p`)

This parameter is used by Minkowski metric, a generalization of the Euclidean and Manhattan distances, denoted by the parameter `p`. When `p = 1`, it is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for `p = 2`. The choice of `p` affects the way distances are calculated between points. It's important to choose a `p` that matches the geometry of the problem, as different values can lead to different performance of the kNN classifier.

#### 3. Metric (`metric`)

The `metric` parameter determines the distance metric to use for the tree. The default metric is `minkowski`, and with `p=2` it is equivalent to the standard Euclidean metric. Other commonly used metrics include:
- `euclidean`: Usual distance between the two vectors (p=2 case for Minkowski).
- `manhattan`: Distance between vectors calculated as the sum of their absolute differences.
- `chebyshev`: Distance measured as the maximum difference between values of the two vectors.

Choosing the right distance metric is essential because it will define how similarity is calculated between instances. Therefore, this choice can affect the accuracy of the classifier.

Each parameter and distance metric plays a crucial role in defining how neighbors are selected and, consequently, how the input data is classified or used in predictions. Proper tuning of these parameters is vital for achieving optimal performance from a kNN model.


**Press ▶ to train the model.**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import ipywidgets as widgets
from IPython.display import display

# Assuming X_train and y_train, X_test, y_test are already defined


# Initialize global variable for KNN classifier
knn = None
y_pred = None

# Function to train KNN and print accuracy
def train_knn(button):
    global n_neighbors, p, metric, knn, y_pred
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, p=p, metric=metric)
    knn.fit(X_train, y_train)
    train_accuracy = knn.score(X_train, y_train)
    test_accuracy = knn.score(X_test, y_test)*100
    y_pred = knn.predict(X_test)
    # Display the output
    with out:
        clear_output(wait=True)
        print(f'The accuracy of the KNN classifier is {test_accuracy:.1f}% on test data')
# Create output widget
out = widgets.Output()

# Function to update KNN parameters
def update_params(change):
    global n_neighbors, p, metric
    n_neighbors = n_neighbors_slider.value
    p = p_slider.value
    metric = metric_dropdown.value

# Create sliders for KNN parameters
n_neighbors_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=15,
    step=1,
    description='n_neighbors:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

p_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='p:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create dropdown for metric selection
metric_dropdown = widgets.Dropdown(
    options=['minkowski', 'euclidean', 'manhattan', 'chebyshev'],
    value='minkowski',
    description='Metric:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create a button to train the model
train_button = widgets.Button(description="Train")

# Link parameter widgets to the update_params function
n_neighbors_slider.observe(update_params, 'value')
p_slider.observe(update_params, 'value')
metric_dropdown.observe(update_params, 'value')

# Set up button click event to trigger the data split
train_button.on_click(train_knn)

# Organize widgets in a vertical box
widgets_box = widgets.VBox([n_neighbors_slider, p_slider, metric_dropdown, train_button, out])

# Display the widgets
display(widgets_box)


**Press ▶ to display the confusion matrix and the classification report.**

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print("Confusion Matrix: ")
print(confusion_matrix(y_test,y_pred))

print("Classification Report: ")
print(classification_report(y_test,y_pred))

## Visualization

In classification tasks, the <span style="color:red">red</span> and <span style="color:blue">blue</span> areas typically represent different classes or categories. The <span style="color:red">red</span> area signifies regions where the classifier predicts one class, while the <span style="color:blue">blue</span> area represents regions where it predicts another class, based on the features of the data points in those regions.


**Press ▶ to visualize the decision boundary.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def plot_decision_regions(X, y, classifier, label_encoder, feature1, feature2, resolution=0.02):
    X = X[[feature1, feature2]]  # Take the selected features for visualization

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ['b', 'r', 'lightgreen', 'gray', 'cyan']
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
    x2_min, x2_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X.loc[y == cl, feature1], y=X.loc[y == cl, feature2],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=label_encoder.inverse_transform([cl])[0])

    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title('KNN Classification with Decision Boundaries')
    plt.legend(loc='upper left')

# Encoding the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Dropdowns for selecting features
feature_selector1 = widgets.Dropdown(
    options=X.columns,
    description='X Axis:',
    disabled=False,
)

feature_selector2 = widgets.Dropdown(
    options=X.columns,
    description='Y Axis:',
    disabled=False,
)

button = widgets.Button(description="Plot")

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        feature1 = feature_selector1.value
        feature2 = feature_selector2.value
        
        if feature1 and feature2:
            knn_model = KNeighborsClassifier()
            knn_model.fit(X_train[[feature1, feature2]], y_train)
            plt.figure(figsize=(10, 5))
            plot_decision_regions(X_test, y_test, knn_model, label_encoder, feature1, feature2)
            plt.show()
        else:
            print("Please select both features")

button.on_click(on_button_click)

display(feature_selector1, feature_selector2, button, output)


### <center>[🏠 Home](../../../../welcomePage.ipynb)</center>